In [1]:
import numpy as np
import pandas as pd
from scipy import linalg

1. Matrix Factorization을 통해서 영화 시청 이력을 기반으로 평점 예측을 해야함
2. 사용해야할 데이터셋 : movies(movieId,genre) / ratings(userId,movieId,rating,timestamp)
3. 학습 데이터: timestamp >= 1104505203 ~ timestamp <= 1230735592
4. 평가 데이터: timestamp >= 1230735600 이후

1. ratings 데이터를 트레이닝 셋과 테스트셋으로 나눔 
2. movieId-userId matrix를 생성
3. matrix factorization(SVD)으로 행렬을 분해 (유저벡터) x (영화벡터)로 나눠야함
4. 다시 복원하면 0들에 값이 채워져있음. 그게 예상하는 값인가? 그 뒤로 어케되는지 전혀 모르겠음 



In [2]:
# data load
df_ratings = pd.read_csv("data/ratings.csv")
df_movies = pd.read_csv("data/movies.csv") 

In [3]:
# train_test_split
df_all = df_ratings.copy()
df_train = df_all[df_all["timestamp"] >= 1104505203]
df_train = df_train[df_train["timestamp"] <= 1230735592]
df_test = df_all[df_all["timestamp"] >= 1230735600]

In [4]:
df_train.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [11]:
# movie preprocessing
df_movies['genres'] = df_movies['genres'].apply(lambda x : x.split("|"))
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df_genre = df_movies.join(pd.DataFrame(mlb.fit_transform(df_movies.pop('genres')),
                         columns = mlb.classes_,
                         index = df_movies.index))

In [13]:
# row = movie, column = genre(featrue)
df_genre

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62418,209157,We (2018),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62419,209159,Window of the Soul (2001),0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
62420,209163,Bad Poems (2018),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
62421,209169,A Girl Thing (2001),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# train set을 전처리 : row = userId column = movieId
df_train_piv = df_train.pivot("userId","movieId","rating")
df_train_piv = df_train_piv.fillna(df_train_piv.mean())
df_train_piv.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,64957,64969,64976,64983,64986,64990,64993,64997,65011,65025
userId,,,,,,,,,,,,,,,,,,,,,
1,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
2,3.500000,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
10,3.500000,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
11,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
12,4.000000,2.919021,2.000000,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0


In [38]:
df_train_piv.index.values

array([     1,      2,     10, ..., 162513, 162516, 162521])

In [24]:
train_movieId = np.array(df_train_piv.columns.tolist())
train_movieId

array([    1,     2,     3, ..., 64997, 65011, 65025])

In [28]:
# df_train_piv를 svd decomposition
U,s,VT = linalg.svd(df_train_piv,compute_uv = True)

In [29]:
r = s.shape[0] - sum(np.allclose(lx,0) for lx in s)
reduced = U[:,:r]*s[:r]@VT[:r,:]
reduced

array([[3.71671402, 2.91902092, 2.79443309, ..., 2.8       , 3.25      ,
        2.        ],
       [3.5       , 2.91902092, 2.79443309, ..., 2.8       , 3.25      ,
        2.        ],
       [3.5       , 2.91902092, 2.79443309, ..., 2.8       , 3.25      ,
        2.        ],
       ...,
       [3.71671402, 2.91902092, 2.79443309, ..., 2.8       , 3.25      ,
        2.        ],
       [4.5       , 2.5       , 0.5       , ..., 2.8       , 3.25      ,
        2.        ],
       [4.        , 2.91902092, 2.79443309, ..., 2.8       , 3.25      ,
        2.        ]])

In [40]:
A = pd.DataFrame(reduced)
A

,0,1,2,3,4,5,6,7,8,9,...,10523,10524,10525,10526,10527,10528,10529,10530,10531,10532
0,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
1,3.500000,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
2,3.500000,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
3,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
4,4.000000,2.919021,2.000000,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27910,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
27911,4.000000,3.500000,3.500000,2.442029,3.500000,3.894462,4.000000,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
27912,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
27913,4.500000,2.500000,0.500000,2.000000,2.686463,4.500000,2.000000,2.864407,2.427419,3.500000,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0


In [41]:
A.index = df_train_piv.index.values
A.columns = train_movieId
A

,1,2,3,4,5,6,7,8,9,10,...,64957,64969,64976,64983,64986,64990,64993,64997,65011,65025
1,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
2,3.500000,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
10,3.500000,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
11,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
12,4.000000,2.919021,2.000000,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162511,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
162512,4.000000,3.500000,3.500000,2.442029,3.500000,3.894462,4.000000,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
162513,3.716714,2.919021,2.794433,2.442029,2.686463,3.894462,2.988117,2.864407,2.427419,3.364015,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0
162516,4.500000,2.500000,0.500000,2.000000,2.686463,4.500000,2.000000,2.864407,2.427419,3.500000,...,3.788889,3.3,2.666667,3.884615,3.0,4.25,3.666667,2.8,3.25,2.0


In [46]:
A.loc[1,296]

5.000000000002372